In [ ]:
############################################################################
#ストラテジーアルゴリズム

def initialize(ctx):
    ctx.logger.debug("initialize() called")
    # 設定    
    # ゴールデンクロスの期間設定
    ctx.long_term = 25 
    ctx.short_term = 5

    # ポートフォリオの何％をオーダーするか
    ctx.target = 0.05

    # 損切りと益出しの水準
    ctx.loss_cut = -0.025
    ctx.plofit = 0.05

    # 銘柄と使う株価データの設定
    ctx.configure(
      channels={          # 利用チャンネル
        "jp.stock": {
          "symbols": [
            "jp.stock.9983", #ファーストリテイリング
            "jp.stock.9984", #ソフトバンク
            "jp.stock.6954", #ファナック
            "jp.stock.9433", #KDDI
            "jp.stock.8028", #ファミリーマート
            "jp.stock.8035", #東京エレクトロン
            "jp.stock.4543", #テルモ
            "jp.stock.6367", #ダイキン
            "jp.stock.6971", #京セラ
            "jp.stock.9735", #セコム
          ],
          "columns": ["close_price_adj",    # 終値(株式分割調整後) 
          ]}})
          
    def _GOLDEN_CROSS(data):
      # 欠損値を埋める
      # cp は pandasのDataFrame（memo 参照）
      cp = data["close_price_adj"].fillna(method="ffill")
      ma_long_term = cp.rolling(window=ctx.long_term, center=False).mean()
      ma_short_term = cp.rolling(window=ctx.short_term, center=False).mean()

      # 長期移動平均を短期移動平均が上抜く
      # 前日は ma_long > ma_short で，
      # 当日は ma_long < ma_short
      buy_signal = (ma_long_term < ma_short_term) & (ma_long_term.shift(1) > ma_short_term.shift(1))

      return {
        "MA_long" : ma_long_term,
        "MA_short" : ma_short_term,
        "Ratio": ma_short_term / ma_long_term - 1,
        "buy:sig": buy_signal,

        }
        
        # シグナル登録
    ctx.regist_signal("GOLDEN_CROSS", _GOLDEN_CROSS)
 
def handle_signals(ctx, date, current):

    # ポジションクローズ
    # ctx.portfolio.positionsは，Keyにシンボル名，Valueに現在の自分のポジション情報を辞書型で持つ，辞書．（memo 参照）
    for (sym,val) in ctx.portfolio.positions.items():
      rtn = val["returns"]

      if rtn < ctx.loss_cut:
        sec = ctx.getSecurity(sym)
        # order_target_percent （memo 参照）
        sec.order_target_percent(0, comment="損切り {:.2%}".format(rtn))

      elif rtn > ctx.plofit:
        sec = ctx.getSecurity(sym)
        sec.order_target_percent(0, comment="益出し {:.2%}".format(rtn))   

     # 買いシグナル
    # currentは該当日の株価やシグナルを含むDataFrame （memo 参照）    
    # DataFrameだと言うことをハッキリさせたかったので df という変数に入れた．
    df = current.copy() 
    # シグナル["buy:sig"]がTrueだけ取得
    df = df[df["buy:sig"]]

    if not df.empty:
      for (sym, val) in df.iterrows(): 
        sec = ctx.getSecurity(sym)
        sec.order_target_percent(ctx.target, comment= "シグナル買 {:.2%}".format(val["Ratio"]))

   